In [1]:
import datetime
import logging
import os
import shutil
import time
from unittest.mock import Mock
import sys

import numpy as np
from io import StringIO
from pathlib import PurePath, Path

from ophyd import (set_cl, EpicsMotor, Signal, EpicsSignal, EpicsSignalRO,
                   Component as Cpt)
from ophyd.utils.epics_pvs import (AlarmSeverity, AlarmStatus)                   

from ophyd.utils.paths import make_dir_tree
from ophyd import (SimDetector, SingleTrigger, Component, Device,
                   DynamicDeviceComponent, Kind, wait)
from ophyd.areadetector.plugins import (ImagePlugin, StatsPlugin,
                                        ColorConvPlugin, ProcessPlugin,
                                        OverlayPlugin, ROIPlugin,
                                        TransformPlugin, NetCDFPlugin,
                                        TIFFPlugin, JPEGPlugin, HDF5Plugin,
                                        register_plugin
                                        # FilePlugin
                                        )
from ophyd.areadetector.base import NDDerivedSignal
from ophyd.areadetector.filestore_mixins import (
    FileStoreTIFF, FileStoreIterativeWrite,
    FileStoreHDF5)

# we do not have nexus installed on our test IOC
# from ophyd.areadetector.plugins import NexusPlugin
from ophyd.areadetector.plugins import PluginBase
from ophyd.areadetector.util import stub_templates
#from ophyd.device import (Component as Cpt, )
from ophyd.signal import Signal
import uuid

from epics import caget, caput, cainfo

from bluesky import RunEngine
from bluesky.utils import ProgressBarManager
from ophyd.sim import motor, det1 
from bluesky.simulators import summarize_plan
from bluesky.plans import count, scan
import bluesky.plan_stubs
from bluesky.callbacks.best_effort import BestEffortCallback

RE = RunEngine({})
RE.waiting_hook = ProgressBarManager()

bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)


0

In [2]:
@register_plugin
class ZMQPlugin(PluginBase): #, version_type='ADZMQ'):
    '''An areadetector plugin class that publishes an NDArray to a ZMQ socket'''
    _default_suffix = 'ZMQ1:'
    _suffix_re = r'ZMQ\d:'
    _plugin_type = 'NDPluginZMQ'
    

In [ ]:
cainfo('13SIM1:cam1:Acquire')
cainfo('13SIM1:ZMQ1:EnableCallbacks')
cainfo('13SIM1:ZMQ1:PluginType_RBV')

In [3]:
class ADZMQStopAndShoot(SingleTrigger, SimDetector):
    zmq1 = Cpt(ZMQPlugin, 'ZMQ1:')

det_zmq = ADZMQStopAndShoot("13SIM1:", name='test')
det_zmq.cam.acquire_time.put(.01)
det_zmq.cam.num_images.put(1)
    
print(det_zmq.zmq1.plugin_type)

print(det_zmq.cam)

EpicsSignalRO(read_pv='13SIM1:ZMQ1:PluginType_RBV', name='test_zmq1_plugin_type', parent='test_zmq1', value='NDPluginZMQ', timestamp=1631244672.710536, auto_monitor=False, string=False)
SimDetectorCam(prefix='13SIM1:cam1:', name='test_cam', parent='test', read_attrs=[], configuration_attrs=['acquire_period', 'acquire_time', 'image_mode', 'manufacturer', 'model', 'num_exposures', 'num_images', 'trigger_mode'])


In [ ]:
caput('13SIM1:cam1:Acquire', 1)
caput('13SIM1:cam1:ArrayCallbacks', 1)
caput('13SIM1:ZMQ1:EnableCallbacks', 1)

In [ ]:
caput('13SIM1:cam1:Acquire', 0)
caput('13SIM1:ZMQ1:EnableCallbacks', 0)

In [6]:
def series_scan_plan(detector, num_frames, scan_type):
    yield from count([detector], num=num_frames, md={'scan_type': scan_type})
       
def tomo_scan_plan(detector,scan_range, num_angles):
    yield from scan([detector], motor, 0, scan_range, num_angles, md={'scan_type': 'tomo'})
    
def sample_scan_plan(detector, num_dark_flat_frames, scan_range, num_angles):
    yield from series_scan_plan(detector, num_dark_flat_frames, 'dark_field')
    yield from series_scan_plan(detector, num_dark_flat_frames, 'flat_field')
    yield from tomo_scan_plan(detector, scan_range, num_angles)

#RE(count(dets, num=20))
#summarize_plan(scan(dets, motor, 0, 180, 180))


In [8]:
RE(sample_scan_plan(det_zmq, 10,180,18), sample_id='test')



Transient Scan ID: 5     Time: 2021-09-10 06:12:28
Persistent Unique Scan ID: '8716755e-45dd-4377-b9ee-110dbee00b2d'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 06:12:28.2 |
|         2 | 06:12:28.2 |
|         3 | 06:12:28.2 |
|         4 | 06:12:28.2 |
|         5 | 06:12:28.3 |
|         6 | 06:12:28.3 |
|         7 | 06:12:28.3 |
|         8 | 06:12:28.3 |
|         9 | 06:12:28.3 |
|        10 | 06:12:28.4 |
+-----------+------------+
generator count ['8716755e'] (scan num: 5)





Transient Scan ID: 6     Time: 2021-09-10 06:12:28
Persistent Unique Scan ID: '3bd4c8a4-fdca-4469-9fe3-a8eddc50989b'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 06:12:28.5 |
|         2 | 06:12:28.5 |
|         3 | 06:12:28.5 |
|         4 | 06:12:28.5 |
|         5 | 06:12:28.6 |
|         6 | 06:12:28.6 |
|         7 | 06:12:28.6 |
|         8 | 06:12:28.6

('8716755e-45dd-4377-b9ee-110dbee00b2d',
 '3bd4c8a4-fdca-4469-9fe3-a8eddc50989b',
 'ab917084-ead6-4a43-9545-d3b55eaa98e2')